# This is the Notebook used for cleaning all the features from grid_features.csv File that contains final feaetures from each Grid

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns


my_data = pd.read_csv("grid_features.csv")

In [ ]:
my_data.head()

In [ ]:
my_data.sample(20)

In [ ]:
stats_df = my_data.describe()

low_count_cols = stats_df.loc['count'] < 10


filtered_df = my_data.drop(columns=low_count_cols[low_count_cols].index)

# Optional: Print removed columns
print("Dropped columns:", list(low_count_cols[low_count_cols].index))


In [ ]:
filtered_df.describe()

In [ ]:
filtered_df.sample(20)

In [ ]:
import osmnx as ox

# Download university data
tags = {"amenity": "parking"}
gdf = ox.features_from_place("Islamabad, Pakistan", tags=tags)

# Extract and save
gdf = gdf[["name", "geometry"]].dropna()
gdf["lat"] = gdf.geometry.centroid.y
gdf["lon"] = gdf.geometry.centroid.x
gdf.to_csv("parking_osm.csv", index=False)

In [ ]:
fi = pd.read_csv("parking_osm.csv")
fi = fi.drop("geometry",axis =1)
fi.head()
fi.to_csv("first_parking.csv")


In [ ]:
import pandas as pd
import googlemaps
import time
import os

# Load your pharmacy dataset
df = pd.read_csv("first_parking.csv")

# Initialize Google Maps API

API_KEY = os.getenv("key")
gmaps = googlemaps.Client(key=API_KEY)

# Prepare columns
df["rating"] = None
df["user_ratings_total"] = None

# Search for each pharmacy using name and location
for idx, row in df.iterrows():
    name = row["name"]
    lat = row["lat"]
    lon = row["lon"]
    
    try:
        # Search with name and location bias
        result = gmaps.places(query=name, location=(lat, lon), radius=100)
        if result["results"]:
            top = result["results"][0]
            df.at[idx, "rating"] = top.get("rating")
            df.at[idx, "user_ratings_total"] = top.get("user_ratings_total")
        
        time.sleep(1)  # Be polite to the API
    except Exception as e:
        print(f"Error at index {idx}: {e}")
        continue

# Save the enriched dataset
df.to_csv("SHOP_PROJECT\All_Shops_Data_CSV\parking_with_ratings.csv", index=False)
print("✅ Done!")
